Import the packages

In [6]:
import os
import sys
# from langchain_community.document_loaders import pypdf
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Weaviate
from langchain.agents import AgentType, initialize_agent
from langchain.schema import SystemMessage
from langchain.embeddings import OpenAIEmbeddings
from typing import List,Union
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv

In [4]:
from langchain_community.document_loaders import PyPDFLoader

load the contract and chunk it

In [7]:
loader = PyPDFLoader('../data/RaptorContract.pdf')
contract = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunk = text_splitter.split_documents(contract)


create an embeder and vectorstore

In [12]:
import weaviate

client = weaviate.Client(embedded_options=EmbeddedOptions)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


Binary /home/kerod/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.7/weaviate-v1.23.7-Linux-amd64.tar.gz
Started /home/kerod/.cache/weaviate-embedded: process ID 41725


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-02-23T02:10:50+03:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-02-23T02:10:50+03:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-02-23T02:10:50+03:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-02-23T02:10:50+03:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-02-23T02:10:50+03:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://12

In [16]:
vectorstore = Weaviate.from_documents(client =  client,
                                      documents= chunk,
                                      embedding=OpenAIEmbeddings(),
                                      by_text = False)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
{"level":"info","msg":"Created shard langchain_70dc1775e9c5474cb1da9f8ec3a1274a_5YbWPc6IDoOo in 3.329098ms","time":"2024-02-23T02:16:40+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-23T02:16:40+03:00","took":362280}


DO a similarity search

In [17]:
query = "Are there any conditions to the closing?"
similarity_search = vectorstore.similarity_search(query)

In [19]:
similarity_search

[Document(page_content='adjustments\ncontemplated\nin\nthis\nAgreement,\nis \nreferred\nto\nherein\nas\nthe\n“\nPurchase\nPrice\n”.\nSection\nII.03\nThe\nClosing\n.\nThe\npurchase\nand\nsale\nof\nthe\nShares\n(the\n“\nClosing\n”)\nshall \ntake\nplace\non\nthe\ndate\nhereof\n(the\n“\nClosing\nDate\n”)\nby\nexchange\nof\ndocuments\nand\nsignatures\n(or \ntheir\nelectronic\ncounterparts).\nThe\nconsummation\nof\nthe\ntransactions\ncontemplated\nby\nthis \nAgreement\nshall\nbe\ndeemed\nto\noccur\nat\n12:01\na.m.\nPacific\nStandard\nTime\non\nthe\nClosing\nDate.\nSection\nII.04\nClosing\nPayments\n.\n(a)\nAt\nthe\nClosing,\nBuyer\nshall\npay,\nor\ncause\nto\nbe\npaid,\nthrough\npayment\nto\nthe \nPaying\nAgent\nwhich\nwill\nin\nturn\nmake\nthe\nfollowing\npayments,\nin\ncash\nby\nwire\ntransfer\nof \nimmediately\navailable\nfunds\n–(a)\nthe\nClosing\nCash\nConsideration\nto\nor\nfor\nthe\naccount\nof\nSellers, \n(b)\nthe\nWarrant\nClosing\nCash\nConsideration\nto\nor\nfor\nthe\naccount\nof\

create a retriver and a q&a pipeline

In [20]:
retriever = vectorstore.as_retriever()

In [21]:
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [22]:
from langchain.chains import RetrievalQA
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [23]:
query2 = "Are there any conditions to the closing?"


In [24]:
qa_stuff.invoke(query2)



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Are there any conditions to the closing?',
 'result': 'Yes, there are conditions to the closing as outlined in the Agreement. The consummation of the transactions contemplated by the Agreement shall occur at 12:01 a.m. Pacific Standard Time on the Closing Date. Additionally, there are specific payments that need to be made at the Closing, such as the Closing Cash Consideration, Warrant Closing Cash Consideration, and Closing Vested Options Cash Consideration. Furthermore, there are assurances, deeds, assignments, transfers, conveyances, and other instruments and papers that may be required to carry out the Contemplated Transactions.'}

In [27]:
langchain.debug = True

In [28]:
qa_stuff.invoke(query)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Are there any conditions to the closing?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "Are there any conditions to the closing?",
  "context": "adjustments\ncontemplated\nin\nthis\nAgreement,\nis \nreferred\nto\nherein\nas\nthe\n“\nPurchase\nPrice\n”.\nSection\nII.03\nThe\nClosing\n.\nThe\npurchase\nand\nsale\nof\nthe\nShares\n(the\n“\nClosing\n”)\nshall \ntake\nplace\non\nthe\ndate\nhereof\n(the\n“\nClosing\nDate\n”)\nby\nexchange\nof\ndocuments\nand\nsignatures\n(or \ntheir\nelectronic\ncounterparts).\nThe\nconsummation\nof\nthe\ntransactions\ncontemplated\nby\nthis \nAgreement\nshall\nbe\ndeemed\nto\noccur\nat\n12:01\na.m.\nPacific\nStandard\nTime\non\nthe\nClosing\nDate.\nSection\nII.04\nClosing\nPayments\n.\n

{'query': 'Are there any conditions to the closing?',
 'result': 'Yes, there are conditions to the closing as outlined in the agreement. The closing involves the exchange of documents and signatures (or electronic counterparts) on the Closing Date. Additionally, there are specific payments to be made at the Closing, such as the Closing Cash Consideration, Warrant Closing Cash Consideration, and Closing Vested Options Cash Consideration. The agreement also mentions the need for certain assurances, deeds, assignments, transfers, conveyances, and other instruments to carry out the Contemplated Transactions.'}

In [29]:
query1 = "How much is the escrow amount?"
qa_stuff.invoke(query1)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "How much is the escrow amount?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "How much is the escrow amount?",
  "context": "with\nthe\nEscrow\nAgreement.\nThe\nEscrow\nAmount\nshall\nbe\nheld\nand, \nsubject\nto\nSection\n2.07,\nreleased\nto\nthe\nCompany\nSecurityholders\nin\naccordance\nwith\nthe \nprovisions\nof\nthe\nEscrow\nAgreement\nwith\nthe\nCompany\nSecurityholders\nbeing\nentitled\nto\nshare\nin \nsuch\nreleased\namounts\nin\naccordance\nwith\ntheir\nPro\nRata\nPercentages.\nFrom\nand\nafter\nthe \nClosing,\nBuyer\nand\nthe\nSellers’\nRepresentative\nwill\ndirect\nthe\nEscrow\nAgent\nto\ndisburse \npayments\nfrom\nthe\nEscrow\nAccount\nin\naccordance\nwith\nthe\npurchase\nprice\nadjustment\nprovisions \

{'query': 'How much is the escrow amount?',
 'result': 'The provided context does not specify a specific amount for the escrow. The escrow amount is determined based on the terms and conditions outlined in the agreement between the parties involved.'}

{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-23T03:27:47+03:00","took":413623}
{"level":"info","msg":"Created shard langchain_a58c905a8d5c4c639ec58a4d16fd34b1_WvxKxRjaQcPs in 17.231375ms","time":"2024-02-23T03:27:47+03:00"}
